# Лабораторна робота 2 з "Асиметричних криптосистем та протоколів"
## Тема: Вивчення криптосистеми RSA та алгоритму електронного підпису; ознайомлення з методами генерації параметрів для асиметричних криптосистем

**Виконали**\
Дигас Богдан, ФІ-03\
Починок Юрій, ФІ-03

In [136]:
import random
rand = random.SystemRandom()

In [137]:

def decomposing_number(n, a):
    exp = n - 1
    while not exp & 1:  # while exp is even
        exp >>= 1  # divide by 2
    if pow(a, exp, n) == 1:
        return True  # number is composite
    while exp < n - 1:
        if pow(a, exp, n) == n - 1:
            return True  # number is composite
        exp <<= 1  # multiply by 2
    return False  # number is probably prime


def miller_rabbin_test(n, k=20):
    for i in range(k):
        a = rand.randrange(1, n - 1)
        if not decomposing_number(n, a):
            return False  # number is composite
    return True  # number is probably prime

def bin_to_dec(bin_n):
    dec_n = 0
    res = 0
    for i in range(len(bin_n)):
        res = bin_n[len(bin_n) - i - 1] * 2 ** i
        dec_n += res
    return dec_n


In [138]:
def generate_bit_seq(n):
    seq = [0]*n
    for i in range(n):
        seq[i] = rand.randint(0, 1)
    return seq


def L20(n):
    seq = generate_bit_seq(20)
    result = [0]*n
    for i in range(20):
        result[i] = seq[i]
    for i in range(20,n):
        result[i] = result[i-3]^result[i-5]^result[i-9]^result[i-20]
    return result

In [139]:
def generate_prime_number(x):
    res = [1,0,0]
    while(miller_rabbin_test(bin_to_dec(res)) == False):
        res = L20(x)
    return res

In [140]:
res = generate_prime_number(256)
print(res)

[1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1]


In [141]:
def generate_better_prime_numbers(n):
    p = bin_to_dec(generate_prime_number(n))
    q = bin_to_dec(generate_prime_number(n))
    
    i = 1
    while(miller_rabbin_test(2*p*i + 1) == False):
        i = i + 1

    better_p = 2*p*i + 1
    
    j = 1
    while(miller_rabbin_test(2*q*j + 1) == False):
        j = j + 1

    better_q = 2*q*j + 1
    
    return better_p, better_q

In [142]:
def RSA_gen_constants():
    p,q = generate_better_prime_numbers(256)
    e = 2**16 + 1
    n = p*q
    phi_n = (p-1)*(q-1)
    d = pow(e,-1,phi_n)
    
    return p,q,n,e,d

In [143]:
class User:
    
    __private_key = None
    __key_pair = None
    
    __k = None

    def __init__(self):
        result = RSA_gen_constants()
        self.__key_pair = result[0], result[1]
        self.public_n = result[2]
        self.public_e = result[3]
        self.__private_key = result[4]
        
    
    def RSA_decrypt(self, C):
        res = pow(C, self.__private_key, self.public_n)
        return res
    
    def RSA_sign(self, M):
        res = pow(M, self.__private_key, self.public_n)
        return res
    
    def Send_Key(self, e_1, n_1):
        while(self.public_n > n_1):
            print("Regenerating keys")
            print(self.public_n," && ", n_1)
            result = RSA_gen_constants()
            self.__key_pair = result[0], result[1]
            self.public_n = result[2]
            self.public_e = result[3]
            self.__private_key = result[4]
        
        k = random.randint(1, self.public_n-1)
        print("initial k, ", hex(k))
        S = pow(k, self.__private_key, self.public_n)
        S_1 = pow(S, e_1, n_1)
        k_1 = pow(k, e_1, n_1)
        
        return k_1, S_1
    
    def Receive_Key(self, k_1, S_1, e, n):
        k = pow(k_1, self.__private_key, self.public_n)
        S = pow(S_1, self.__private_key, self.public_n)
        
        if k == pow(S, e, n):
            __k = k
            print("k is authentic, ", hex(k))
        else:
            print("k is not authentic, try again", hex(k))

In [144]:
A = User()
B = User()

def RSA_encrypt(m,e,n):
    res = pow(m,e,n)
    return res

def RSA_verify(S, M, e, n):
    res = pow(S,e,n)
    return res == M


In [145]:
print(hex(A.public_n))

0x7c77ede886abef195f2792b86dd4cfe3f87511d4602d12366f0fcac172ee09472b1631ba15133ead59cf154e99744f7bbe4cf4dfd49892a10b4332170e9357fc3d59


## Server Key

In [146]:
server_key = int("A50E89F2E5BB9EE223D6D793584A035708E19079C9AC9D6778A97AF2E4F8D44B", 16)
server_e = 2**16+1

## Encryption

In [147]:
M = int("1234", 16)
print(hex(A.public_n))
print(hex(A.public_e))
print(hex(RSA_encrypt(M, server_e, server_key)))

0x7c77ede886abef195f2792b86dd4cfe3f87511d4602d12366f0fcac172ee09472b1631ba15133ead59cf154e99744f7bbe4cf4dfd49892a10b4332170e9357fc3d59
0x10001
0x24676dec62c0bddc092bd1f89024ec0f2a2ee0fceb7df41bc4169578b16cad1a


## Decryption

In [148]:
C = int("016912CBD39A37DD58FE1A1043D486EAF6BE", 16)
print(hex(A.RSA_decrypt(C)))

0x61aabba0e33df4e95bdb28ee2b559ec41cb4b56f948d32d3aefe9f6c8d4f6976b4c48784657bd25d1133fcb4f7dd5386eedf07548e6254349deef47c4cbec88bdcef


## Signature

In [149]:
M = int("12345", 16)
server_S = int("58E8A4BF4029D91D14362C9352FADBE39B287ED108D7BC7E76A437895C4B4130",16)
print(RSA_verify(server_S, M, server_e, server_key))

True


## Verification

In [150]:
M = int("12345", 16)
print(hex(A.RSA_sign(M)))
print(hex(A.public_n))
print(hex(A.public_e))

0x5b85c85817b647fadf34fd3101dc5d3774bdafcd17979cf65a187172fdbf5348b41e80149fd1d157c3698bdac9fc8e6350fd2dc1ccffb58b675f5ebb048bdb45d204
0x7c77ede886abef195f2792b86dd4cfe3f87511d4602d12366f0fcac172ee09472b1631ba15133ead59cf154e99744f7bbe4cf4dfd49892a10b4332170e9357fc3d59
0x10001


## Send key


In [155]:
server_key = int("82020B15F140019B0DC1D29AD49C3E49F20E40B528C86334877E6FBBC8C599A7", 16)
print(hex(A.public_n))
print(hex(A.public_e))
server_encrypted_key = int("0585244AC40EB85C5FEEA7B985282C3736393B52D79C0FCD3011F58C8E5469A7C984BE7359CABF484928EDB134D5224A1414A99FA5DFF0AF9305F599EA1E39B01645", 16)
server_encrypted_signature = int("020CE94F4FB9C991E2F263F7E595AE113F0B6FD5FC34680B97C7FFADC374B1EBECEF8C84C3947E9C48D3A13AAA7490419D3EFC8430C3E4565B4F267E1CB2C581F421",16)
A.Receive_Key(server_encrypted_key,server_encrypted_signature, A.public_e, server_key)

0x6c65000b3dc3dc4451604f007c4bb9a612b95951d2ad17744f11511213f0f53ef52711fc03fd129b7db016f26e1ddf4ce90b728102d648f6035791be8c12335eab7
0x10001
k is authentic,  0x28323ff8155b3a4d


## Recieve Key

In [152]:
server_key = int("880A90A3D37D5349E5C2C76ABF19A6D9F9357007F7A43F4C4F1FDC8FA178144939F69ED731E68E06176184E1E6B6AB5662E8A2564965CCBB90250E955FED820B", 16)
server_e = 2**16+1
k_1, s_1 = A.Send_Key(server_e, server_key)
print(hex(k_1))
print(hex(s_1))
print(hex(A.public_n))
print(hex(A.public_e))

Regenerating keys
427225444340735124973392889413858760089933841370237088119032649711097469281035223310652514163229708224867230047331708141029464315927838777357751766217695968601  &&  7125059423474485796427780114108710069718823953327767324525465666096242746697807717781752420222338123161243304063446945822845567401467684540227569499079179
Regenerating keys
21749537016226742375351305454095702556508912389802923034129480232333154268396878118224043711339543599679351112866759229865827631696109045499855783420810039681  &&  7125059423474485796427780114108710069718823953327767324525465666096242746697807717781752420222338123161243304063446945822845567401467684540227569499079179
Regenerating keys
3440534774324554428483524805940501273160135305291357236840515783468302802394317931614931390492900723808153830831720975690292382104409106918607497940646769337  &&  712505942347448579642778011410871006971882395332776732452546566609624274669780771778175242022233812316124330406344694582284556740146768454022756

KeyboardInterrupt: 